# Project 1: Mid-Term - ETL Processing

#### Import the Necessary Libraries

In [8]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [6]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.38
Running PyMongo Version: 4.11.2


In [47]:
host_name = "localhost"
port = "3306"
user_id = "bhowe"
pwd = "Daufuskie1"

src_dbname = "adventureworks"
dst_dbname = "adventureworks_dw"

In [48]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [9]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

#### Extract Data from the Source Database Tables

In [12]:
# Customers
sql_customers = "SELECT * FROM adventureworks.customer;"
df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
df_customers.head(2)

,CustomerID,TerritoryID,AccountNumber,CustomerType,rowguid,ModifiedDate
0,1,1,AW00000001,S,b'^\xe9Z?}\xb8\xedJ\x95\xb4\xc3yz\xfc\xb7O',2004-10-13 11:15:07
1,2,1,AW00000002,S,b'W\xf6R\xe5\xaf\xa9}J\xa6E\xc4)\xd6\xe0$\x91',2004-10-13 11:15:07


In [14]:
# Employees
sql_employees = "SELECT * FROM adventureworks.employee;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head(2)

,EmployeeID,NationalIDNumber,ContactID,LoginID,ManagerID,Title,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,14417807,1209,adventure-works\guy1,16.0,Production Technician - WC60,1972-05-15,M,M,1996-07-31,b'\x00',21,30,b'\x01',b'J\xd0\xe1\xaa7\xc2tI\xb4\xd5\x93RGsw\x18',2004-07-31
1,2,253022876,1030,adventure-works\kevin0,6.0,Marketing Assistant,1977-06-03,S,M,1997-02-26,b'\x00',42,41,b'\x01',b'@\x02H\x1b\xc0\x95\x0fA\xa7\x17\xeb)\x94<\x8...,2004-07-31


In [15]:
# Products
sql_products = "SELECT * FROM adventureworks.product;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xb7\x15Bi\xf7\x08\rL\xac\xb1\xd74\xbaD\xc0\...,2004-03-11 10:01:36
1,2,Bearing Ball,BA-8327,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b' <\xaeX:OIG\xa7\xd4\xd5h\x80l\xc57',2004-03-11 10:01:36


In [16]:
# Vendors
sql_vendors = "SELECT * FROM adventureworks.vendor;"
df_vendors = get_dataframe(user_id, pwd, host_name, src_dbname, sql_vendors)
df_vendors.head(2)

,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1,INTERNAT0001,International,1,b'\x01',b'\x01',None,2002-02-25
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,b'\x01',b'\x01',None,2002-02-17


#### Create the Date Dimension Table
Executed the script from earlier Lab 2c to create and populate a date dimension table (dim_date) that will be extracted from MySQL.

#### Perform Necessary Transformations

In [ ]:
# Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['rowguid']
df_customers.drop(drop_cols, axis=1, inplace=True)

# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))

# Display the first 2 rows of the dataframe to validate your work
df_customers.head(2)

,customer_key,CustomerID,TerritoryID,AccountNumber,CustomerType,ModifiedDate
0,1,1,1,AW00000001,S,2004-10-13 11:15:07
1,2,2,1,AW00000002,S,2004-10-13 11:15:07


In [19]:
# Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['SalariedFlag','CurrentFlag','rowguid']
df_employees.drop(drop_cols, axis=1, inplace=True)

# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))

# Display the first 2 rows of the dataframe to validate your work
df_employees.head(2)

,employee_key,EmployeeID,NationalIDNumber,ContactID,LoginID,ManagerID,Title,BirthDate,MaritalStatus,Gender,HireDate,VacationHours,SickLeaveHours,ModifiedDate
0,1,1,14417807,1209,adventure-works\guy1,16.0,Production Technician - WC60,1972-05-15,M,M,1996-07-31,21,30,2004-07-31
1,2,2,253022876,1030,adventure-works\kevin0,6.0,Marketing Assistant,1977-06-03,S,M,1997-02-26,42,41,2004-07-31


In [41]:
# Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['MakeFlag','FinishedGoodsFlag','SellEndDate','DiscontinuedDate','SizeUnitMeasureCode','WeightUnitMeasureCode','rowguid']
df_products.drop(drop_cols, axis=1, inplace=True)

# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))

# Display the first 2 rows of the dataframe to validate your work
df_products.head(2)

,product_key,ProductID,Name,ProductNumber,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,ModifiedDate
0,1,1,Adjustable Race,AR-5381,None,1000,750,0.0,0.0,None,NaN,0,None,None,None,NaN,NaN,1998-06-01,2004-03-11 10:01:36
1,2,2,Bearing Ball,BA-8327,None,1000,750,0.0,0.0,None,NaN,0,None,None,None,NaN,NaN,1998-06-01,2004-03-11 10:01:36


In [45]:
# Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['PreferredVendorStatus','ActiveFlag','PurchasingWebServiceURL']
df_vendors.drop(drop_cols, axis=1, inplace=True)

# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_vendors.insert(0, "vendor_key", range(1, df_vendors.shape[0]+1))

# Display the first 2 rows of the dataframe to validate your work
df_vendors.head(2)

,vendor_key,VendorID,AccountNumber,Name,CreditRating,ModifiedDate
0,1,1,INTERNAT0001,International,1,2002-02-25
1,2,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,2002-02-17


#### Load the Transformed DataFrames into "adventuresworks_dw" by Creating New Tables

In [46]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_employees', df_employees, 'employee_key'),
          ('dim_products', df_products, 'product_key'),
          ('dim_vendors', df_vendors, 'vendor_key')]

In [49]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

# Next Steps

In [ ]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Working Databases

In [1]:
mysql_args = {
    "uid" : "bhowe",
    "pwd" : "Daufuskie1",
    "hostname" : "localhost",
    "dbname" : "adventureworks2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "nkr7fg",
    "password" : "Daufuskie1",
    "cluster_name" : "ds2002",
    "cluster_subnet" : "zsrbu",
    "cluster_location" : "atlas", # "local"
    "db_name" : "northwind_purchasing"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [6]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [4]:
dst_dbname = "adventture"

conn_str = f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

NameError: name 'create_engine' is not defined

In [9]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"suppliers" : 'northwind_suppliers.json',
              "invoices" : 'northwind_invoices.json',
              "purchase_orders" : 'northwind_purchase_orders.json',
              "inventory_transactions" : 'northwind_inventory_transactions.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)    

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\puckh\\Desktop\\Brice Documents\\Fourth Year\\DS Systems 2002\\DS-2002\\Projects\\data\\northwind_suppliers.json'